In [2]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math

import tensorflow as tf

# Lecture 12
## 3 ML을 이용한 회귀분석 원리


In [3]:
# x, y 데이터 정의
x_data=[1,2,3,4,5]
y_data=[1,2,3,4,5]

In [4]:
# W: 텐서플로우의 베리어블로 정의. 
# 왜 베리어블로 정의하는 지는 저번 시간에 이야기 함: 미분할 때는 값이 고정된게 아니라 조금씩 이동하기 때문
W=tf.Variable(2.0) # initial value
b=tf.Variable(0.5) # initial value

In [5]:
hypothesis=W*x_data+b
print(hypothesis)

tf.Tensor([ 2.5  4.5  6.5  8.5 10.5], shape=(5,), dtype=float32)


In [6]:
# [ 2.5  4.5  6.5  8.5 10.5] : 초기값에서 나온 가설의 값, 회귀직선의 값

In [7]:
# cost function 정의
cost=tf.reduce_mean(tf.square(hypothesis-y_data))
print(cost)

tf.Tensor(14.25, shape=(), dtype=float32)


In [8]:
# 14.25 =((2.5-1)**2+ () + .... )/5

In [9]:
cost.numpy()

14.25

In [10]:
# 초기값 주기
tf.random.set_seed(0)
W=tf.Variable(tf.random.normal([1]), name='weight') # [1]: random.normal에서 하나를 끄집어내
b=tf.Variable(tf.random.normal([1]), name='bias')
learning_rate=tf.Variable(0.01)

In [11]:
# 1001번 반복하도록 한다
for i in range(1000+1):
    with tf.GradientTape() as tape:
        hypothesis=W*x_data+b
        cost=tf.reduce_mean(tf.square(hypothesis-y_data))
    W_grad, b_grad=tape.gradient(cost,[W,b])
    W.assign_sub(learning_rate*W_grad) # _sub: 빼준다, 이 라인이 그 알파써서 W 옮기는 식 쓴거임
    b.assign_sub(learning_rate*b_grad)
    if i%200==0: # 1000번 반복 중에 너무 많으니까 200번 마다 보여줘라
        print(cost.numpy(),W.numpy(),b.numpy())

7.282724 [1.3346161] [1.0148789]
0.034686986 [0.8794935] [0.43506676]
0.008950316 [0.9387867] [0.2209997]
0.0023094572 [0.9689056] [0.11226072]
0.0005959122 [0.984205] [0.05702474]
0.00015376188 [0.9919767] [0.02896672]


In [12]:
# ... [] [] : costfunction의 값, w의 값, b의 값
# 0.00015376188 [0.9919767] [0.02896672]
# 거의 0, 거의 1, 거의 0
# 1000번 실행했더니 예상한 방향으로 갔다

In [13]:
# 위 코드 중요함

In [14]:
# x 변수가 여러개면 위에 x_data를 어떻게 고치지가 고민

# Lecture 13 회귀분석과 머신러닝(3)
## 1 머신러닝을 이용한 주택가격(지수) 추정
- LTV, DTI 등..

In [15]:
df=pd.read_csv('c:/temp/HPI_SEOUL.csv',encoding='CP949')
df.head()

,YYM1,CITY,CITY_CODE,GU_NAME,GU_CODE2,API_TT0,BaseRate,MoRate,FREQ,A_PSUM,...,LTV,DTI,MV_APT,JAEGO,BUNYANG,N_BUNYANG,IND_INDEX,JTP,LOAN_T,MON
0,200603,서울특별시,11,종로중구,1111,75.245608,4.00,5.46,71,2.559029,...,42.912677,93.255607,4.891421,110,0,277,109.907,48.372,104.381241,3
1,200606,서울특별시,11,종로중구,1111,81.009183,4.25,5.48,58,1.774481,...,43.944172,94.827586,4.891421,0,0,277,99.469,47.454,107.564252,6
2,200609,서울특별시,11,종로중구,1111,81.208640,4.50,5.77,90,2.898839,...,44.425621,94.000000,4.891421,0,0,14,123.203,47.670,110.666410,9
3,200612,서울특별시,11,종로중구,1111,88.068940,4.50,5.88,99,3.181162,...,50.340131,95.151515,4.891421,0,136,6,128.718,43.811,101.843742,12
4,200703,서울특별시,11,종로중구,1111,86.507762,4.50,6.20,68,1.955622,...,50.951451,48.890836,6.083816,0,112,0,110.893,42.987,107.877226,3


- ex. API_TT0: 100을 기준으로 생각하며됨, '주택가격지수'
- 분기마다 측정한 가격지수임. yym1 보셈
- BaseRate: '기준금리' -> 모기지금리가 결ㅈㅇ되고 -> 주택가격도 결정되니까
- IND_INDEX: industry index, 경기 보는것, 오르면 호경기
- LTV: 집 최대 몇퍼까지 빚낼 수 있는지
- DTI: 소득대비 빚낼 수 있는 정도

In [16]:
df1=df[['API_TT0','GU_CODE2','BaseRate','IND_INDEX','LTV','DTI','FREQ']]
# FREQ: frequency (3개월간)주택거래량
df2=df1.copy()

In [17]:
# 과거 값을 변수로 생성해야함 . shift 이용
# 주의!! : shift(1) 할 때, 그 구 내에서만 전걸로 가져오고, 다른 구로 넘어가면 그 구안에서 또 하나씩 과거 껄 가져와야 함
# groupby 로 씀 그걸
# groupby('GU_CODE2'): 구 코드 별로 그룹바이 하셈

In [18]:
df2['BaseRate1']=df2.groupby('GU_CODE2')['BaseRate'].shift(1)
df2['IND_INDEX1']=df2.groupby('GU_CODE2')['IND_INDEX'].shift(1)
df2['LTV2']=df2.groupby('GU_CODE2')['LTV'].shift(2)
df2['DTI2']=df2.groupby('GU_CODE2')['DTI'].shift(2)
df2.head()

,API_TT0,GU_CODE2,BaseRate,IND_INDEX,LTV,DTI,FREQ,BaseRate1,IND_INDEX1,LTV2,DTI2
0,75.245608,1111,4.00,109.907,42.912677,93.255607,71,NaN,NaN,NaN,NaN
1,81.009183,1111,4.25,99.469,43.944172,94.827586,58,4.00,109.907,NaN,NaN
2,81.208640,1111,4.50,123.203,44.425621,94.000000,90,4.25,99.469,42.912677,93.255607
3,88.068940,1111,4.50,128.718,50.340131,95.151515,99,4.50,123.203,43.944172,94.827586
4,86.507762,1111,4.50,110.893,50.951451,48.890836,68,4.50,128.718,44.425621,94.000000


In [19]:
# 뒤에 변수 4개 추가 됨

In [20]:
# 보면 결측치가 있음. 결측치 삭제한 데이터를 만들자
df3=df2.dropna()

In [21]:
df3.head()

,API_TT0,GU_CODE2,BaseRate,IND_INDEX,LTV,DTI,FREQ,BaseRate1,IND_INDEX1,LTV2,DTI2
2,81.208640,1111,4.5,123.203,44.425621,94.000000,90,4.25,99.469,42.912677,93.255607
3,88.068940,1111,4.5,128.718,50.340131,95.151515,99,4.50,123.203,43.944172,94.827586
4,86.507762,1111,4.5,110.893,50.951451,48.890836,68,4.50,128.718,44.425621,94.000000
5,91.769299,1111,4.5,102.227,51.251565,48.351863,48,4.50,110.893,50.340131,95.151515
6,92.079847,1111,5.0,108.332,52.053270,48.055339,55,4.50,102.227,50.951451,48.890836


In [22]:
# OLS로 다중회귀모형 추정하기
housing_model=ols('API_TT0 ~ BaseRate1+IND_INDEX1+LTV2+DTI2',data=df3).fit()
print(housing_model.summary())

                            OLS Regression Results                            
Dep. Variable:                API_TT0   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.366
Method:                 Least Squares   F-statistic:                     160.4
Date:                Fri, 14 Jun 2024   Prob (F-statistic):          3.75e-108
Time:                        19:18:46   Log-Likelihood:                -3856.6
No. Observations:                1104   AIC:                             7723.
Df Residuals:                    1099   BIC:                             7748.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.1081      3.594     20.065      0.0

In [23]:
# BaseRate1     -2.9000: 금리가 높아지면 주택가격이 떨어진다. 라는 뜻
# IND_INDEX1     0.1311 : 경기가 좋아지면 주택가격도 높아진다
# LTV2           0.3556 : LTV 커지면 주택가격 높아진다
# DTI2          -0.0526 : DTI 커지면 주택가격 낮아진다

In [24]:
# 경사하강법을 써서 해보자

# x, y 데이터 정의
# 데이터프레임에서 ols 처럼 그냥 가져와서 쓸순 없고
# 저기서 가져와서 array 로 만들어 정의해야함 : .to_numpy()
X=df3[['BaseRate1','IND_INDEX1','LTV2','DTI2']].to_numpy()
y=df3[['API_TT0']].to_numpy()
# tensor type 정해줘야함
X=X.astype('float32')
y=y.astype('float32')

In [25]:
X.shape # x변수 4개이고, 1104개의 행으로 구성되어있구나

(1104, 4)

In [26]:
k=X.shape[1] #k=4 라는 걸 정의한 것

In [27]:
# 초기값 주기
tf.random.set_seed(0)
W=tf.Variable(tf.random.normal([k,1]), name='weight') # [k,1]: x변수가 4개 이니까
b=tf.Variable(tf.random.normal([1]), name='bias')
learning_rate=tf.Variable(0.01)

In [28]:
# 1001번 반복하도록 한다
for i in range(1000+1):
    with tf.GradientTape() as tape:
        hypothesis=tf.matmul(X,W)+b # X변수가 4개임. 행렬곱임. : tf.matmul(X,W) 로 써야함. 행렬곱이라 dimension이 맞아야 해서 () 안에 X가 먼저 나오는 거임 
        cost=tf.reduce_mean(tf.square(hypothesis-y))
    W_grad, b_grad=tape.gradient(cost,[W,b])
    W.assign_sub(learning_rate*W_grad) # _sub: 빼준다, 이 라인이 그 알파써서 W 옮기는 식 쓴거임
    b.assign_sub(learning_rate*b_grad)
    if i%200==0: # 1000번 반복 중에 너무 많으니까 200번 마다 보여줘라
        print(cost.numpy(),W.numpy(),b.numpy())

19553.852 [[  8.847416]
 [288.41885 ]
 [162.8633  ]
 [181.27948 ]] [3.8239536]
nan [[nan]
 [nan]
 [nan]
 [nan]] [nan]
nan [[nan]
 [nan]
 [nan]
 [nan]] [nan]
nan [[nan]
 [nan]
 [nan]
 [nan]] [nan]
nan [[nan]
 [nan]
 [nan]
 [nan]] [nan]
nan [[nan]
 [nan]
 [nan]
 [nan]] [nan]


In [30]:
import qgrid
qgrid_widget=qgrid.show_grid(df3,show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…